In [13]:
from pruneshift.networks import create_network
from pruneshift.datamodules import ShiftDataModule
from pruneshift.utils import safe_ckpt_load
import torch

CKPT_PATH = "/misc/lmbraid19/hoffmaja/experiments/2021-03-22/10-34-58/checkpoint/last.ckpt"

In [67]:
net = create_network("imagenet", "mnasnet1_0", 100, pretrained=True, imagenet_subset=True)



In [16]:
# state_dict = torch.load(CKPT_PATH)["state_dict"]

In [19]:
net.load_state_dict(state_dict)

<All keys matched successfully>

In [65]:
data = ShiftDataModule("imagenet", batch_size=128, root="/misc/scratchSSD2/datasets/ILSVRC2012-100")

In [66]:
data.setup("fit")

In [68]:
idx, x, y = next(iter(data.train_dataloader()))

In [71]:
print("True labels:")
print(y)
print("Train mode:")
net.train()
print(net(x).argmax(1))
print("Eval mode:")
net.eval()
print(net(x).argmax(1))
print("Dropout Eval mode:")
net.train()
net.network.classifier[0].train()
print(net(x).argmax(1))
print("Batch norm Eval mode:")
# net.load_state_dict(state_dict)
net.train()
for module in net.modules():
    if isinstance(module, torch.nn.BatchNorm2d):
        module.eval()
print(net(x).argmax(1))

for name, module in net.named_modules():
    if not isinstance(module, torch.nn.BatchNorm2d):
        continue
    # net.load_state_dict(state_dict)
    net.train()
    module.eval()
    print(f"Batch norm {name} Eval mode:")
    print(net(x).argmax(1))

True labels:
tensor([71, 55, 93, 88, 99,  0, 43, 53, 17, 36, 70, 83, 59, 89, 95, 37,  0, 47,
        14, 48, 54, 54, 89, 41, 39, 39, 69, 85, 51, 48, 97, 80, 39, 41, 29, 24,
        55,  8, 17, 84, 19, 26, 24, 14, 78, 18, 71, 23, 32,  3, 60, 65, 20, 42,
        55, 20, 24, 62, 16, 53, 74,  3,  8, 27, 51,  6, 64, 82, 30, 89, 85, 52,
        41, 50, 48,  2, 11, 77, 75, 33, 48, 48, 11,  3, 65, 61, 95, 36, 89, 96,
        43, 84, 44, 39,  6, 15, 46, 30, 78, 56, 17, 83,  4, 54, 85, 33, 71, 32,
        33, 58, 15, 86, 35, 49, 83, 39, 89, 80, 80, 84, 87,  3, 38, 97,  9,  7,
        57, 77])
Train mode:
tensor([71, 55, 93, 88, 99,  0, 43, 94, 18, 36, 98, 83, 59, 89, 95, 37,  0, 47,
        14, 48, 54, 54, 89, 41, 39, 39, 69, 85, 51, 48, 97, 80, 64, 41, 29, 24,
        66,  8, 17, 84, 19, 26, 33, 14, 78, 18, 71, 23, 32,  3, 41, 65, 20, 41,
        55, 20, 33, 62, 29, 77, 63,  3,  8, 27, 51,  6, 64, 82, 30, 89, 18, 52,
        41, 50, 48,  2, 11, 73, 75, 34, 48, 63, 11,  3, 65, 61, 95, 36, 89, 96

In [56]:
show_results(net, x, y)

True labels:
Train mode:
tensor([28, 79, 54, 45, 53, 86,  2, 53, 80, 60, 30, 13, 95, 24, 90, 21, 23, 56,
        86, 93, 59, 94, 44, 39, 63, 52, 53,  5, 54, 33, 97,  7])


In [28]:
pred = net(x).argmax(1)

In [29]:
pred.classifier

tensor([28, 44, 54, 45,  1, 86,  2, 57, 65, 60, 30, 13, 95, 24, 90, 21, 23, 56,
        86, 93, 59, 94, 24, 39, 63, 52, 53,  5, 54, 33, 97,  7])

In [30]:
y

tensor([99, 93, 54, 83, 69, 32,  5,  1,  4, 61, 99, 81, 12, 77,  5, 21, 93, 29,
         7, 93, 59, 76, 67, 39, 42, 39, 53,  5, 54, 29, 36, 24])

In [31]:
pred == y

tensor([False, False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False,  True,
         True, False, False,  True, False, False,  True,  True,  True, False,
        False, False])

In [47]:
net = net.train()

In [53]:
pred = net(x).argmax(1)

In [54]:
pred

tensor([28, 44, 54, 45,  1, 86,  2, 53, 80, 60, 30, 13, 68, 24, 90, 21, 23, 56,
        66, 93, 59, 94, 24, 39, 63, 52, 53, 20, 54, 33, 97,  7])

In [52]:
net.classifier[0].train()

Dropout(p=0.2, inplace=True)